# SRI Services

## 1. Biolink Lookup

The Biolink Model defines a set of common concepts for use in Translator.  These include semantic types for entities, as well as the relations between them.  These concepts are organized into an inheritance hierarchy capturing different granularities of description. Furthermore, each concept contains metadata relating the concept to ontologies.

The Biolink Model is formally maintained in Github [here](http://github.com/biolink/biolink-model/).  The Biolink Lookup [service](https://bl-lookup-sri.renci.org/apidocs/) provides a computational interface to the model, including access to previous versions.   When the service is deployed, it queries the Biolink Github repository, and updates itself to access the latest version.

In [ ]:
import requests
import json

### Concept Lookup

Biolink concepts can be retrieved to inspect their properties:

In [ ]:
response = requests.get('https://bl-lookup-sri.renci.org/bl/chemical_substance')
print( json.dumps(response.json (),indent = 2 ))

### Versions

Because the service allows access to multiple versions of Biolink, this function returns the list of versions that can passed as a parameter to subsequent calls.  Here, there are 3 versions: 
* **latest** which always points to the current master branch.
* **1.2.1** An older version of the Biolink model
* **custom** which is a special tag pointing at a branch of the model.

In [ ]:
response = requests.get('https://bl-lookup-sri.renci.org/versions/')
print(json.dumps(response.json(),indent=2))

## 2. Node Normalization

[Node normalization](https://nodenormalization-sri.renci.org/apidocs) takes a CURIE, and returns:

* The preferred CURIE for this entity
* All other known equivalent identifiers for the entity
* Semantic types for the entity as defined by the [Biolink Model](https://biolink.github.io/biolink-model/)

The data currently served by Node Normalization is created by the prototype project [Babel](https://github.com/TranslatorIIPrototypes/Babel), which attempts to find identifier equivalences, and makes sure that CURIE prefixes are Biolink Model compliant.  NodeNormalization, however, is independent of Babel and as improved identifier equivalence tools are developed, their results can be easily incorporated.

In this example, `get_normalized_node` is called with a MeSH identifier.   MeSH contains many different semantic types, but the service correctly identifies the term.

In [ ]:
result = requests.get('https://nodenormalization-sri.renci.org/get_normalized_nodes',
                     params={'curie':"MESH:D014867"})
print( json.dumps( result.json(), indent = 2))

## 3. Edge Normalization

The [Biolink Model](https://biolink.github.io/biolink-model/) defines allowed predicates in the Translator ecosystem.  Ingesting data from arbitrary sources requires mapping predicates in those sources to Translator predicates. 
 
The [EdgeNormalization Service](https://edgenormalization-sri.renci.org/apidocs) can find predicates if they have an exact mapping in the model.  The EdgeNormalization service takes this a step further, and attempts to find the best match to a Biolink predicate, even if there is not an explicit mapping. 

### Find BioLink Predicate for unmapped term

EdgeNormalization can also return the best Biolink predicate for an unmapped term.  At the moment, this functionality is limited to terms from Relation Ontology (RO).   Here, we will begin with `RO:0002354 (formed as a result of)`. Checking the [Biolink Lookup Service](https://bl-lookup-sri.renci.org/apidocs/) we can see that there is no direct mapping:

In [ ]:
response=requests.get('https://bl-lookup-sri.renci.org/uri_lookup/RO:0002354')
print(response.status_code)
props = response.json()
print(props)

However, if we call EdgeNormalization, we are returned a suitable Biolink Model predicate:

In [ ]:
response = requests.get('https://edgenormalization-sri.renci.org/resolve_predicate',
                        params={'predicate':['RO:0002354']})
print( json.dumps(response.json(), indent=2))

This particular term was returned because it is a superclass of the original term that also maps directly to Biolink:

![alt text](RelationOntology.png "Relations")

## 4. Name Resolution

Translator tools use strong identifiers for concepts, such as compact URIs (curies) from an ontology.  The [NameResolution sevice](https://nameresolution-sri.renci.org) allows lookup of these curies based on a lexical name.  The identifiers retrieved may then be used as input to various Translator tools.

Suppose that a user has the text string "diabetes" and we want to turn it into an identifier.  There are many concepts that might be appropriate.  Perhaps they are looking for the identifier for the disease "Diabetes Mellitus".  Or perhaps they are looking for a subtype such as "Type 2 Diabetes Mellitus".  Or perhaps they are looking for "diabetes drugs".  This service searches the lexical synonyms of concepts and returns those identifiers as well as the full set of synonyms for each of the returned identifiers.

In [ ]:
nr_url = 'http://robokop.renci.org:2434/lookup'

In [ ]:
params = {'string':'type 2 diabetes'}
response = requests.post(nr_url,params=params)
print(json.dumps(response.json(),indent=1))